# PyTorch Transfer Learning

 1.   Freezing all the layers except the final one
 2.   Freezing the first few layers
 3.   Fine-tuning the entire network.

The following pre-trained models are available on PyTorch

 *   resnet18, resnet34, resnet50, resnet101, resnet152
 *   squeezenet1_0, squeezenet1_1
 *   Alexnet
 *   inception_v3
 *   Densenet121, Densenet169, Densenet201
 *   Vgg11, vgg13, vgg16, vgg19, vgg11_bn. vgg13_bn, vgg16_bn, vgg19_bn

In [1]:
import sys
sys.version

'3.6.7 (default, Oct 22 2018, 11:32:17) \n[GCC 8.2.0]'

In [0]:
# http://pytorch.org/
from os.path import exists
from wheel.pep425tags import get_abbr_impl, get_impl_ver, get_abi_tag
platform = '{}{}-{}'.format(get_abbr_impl(), get_impl_ver(), get_abi_tag())
cuda_output = !ldconfig -p|grep cudart.so|sed -e 's/.*\.\([0-9]*\)\.\([0-9]*\)$/cu\1\2/'
accelerator = cuda_output[0] if exists('/dev/nvidia0') else 'cpu'

!pip install -q http://download.pytorch.org/whl/{accelerator}/torch-0.4.1-{platform}-linux_x86_64.whl torchvision
#import torch

In [3]:
!pip install Pillow
!pip install image

In [0]:
%matplotlib inline

In [0]:
from __future__ import print_function, division

import torch
import torch.nn as nn
import torch.optim as optim
from torch.optim import lr_scheduler
import numpy as np
import torchvision
from torchvision import datasets, models, transforms
import matplotlib.pyplot as plt
import time
import os
import copy

plt.ion()   # interactive mode

In [6]:
import torch
print(torch.__version__)

0.4.1


In [7]:
import tensorflow as tf
tf.test.gpu_device_name()

'/device:GPU:0'

In [8]:
torch.cuda.is_available()

True

### Get Data

In [0]:
DATASET_ZIP_FILE = 'flower_data.zip'

In [0]:
from zipfile import ZipFile
files = os.listdir()
if not DATASET_ZIP_FILE in files:
  !curl --header 'Host: s3.amazonaws.com' --user-agent 'Mozilla/5.0 (X11; Ubuntu; Linux x86_64; rv:63.0) Gecko/20100101 Firefox/63.0' --header 'Accept: text/html,application/xhtml+xml,application/xml;q=0.9,*/*;q=0.8' --header 'Accept-Language: en-US,en;q=0.5' --header 'Upgrade-Insecure-Requests: 1' 'https://s3.amazonaws.com/content.udacity-data.com/courses/nd188/flower_data.zip' --output 'flower_data.zip'  
  '''
  !pip install kaggle
  from google.colab import files
  files.upload()
  !mkdir -p ~/.kaggle
  !cp kaggle.json ~/.kaggle/
  #this permissions change avoids a warnong on Kaggle tool startup
  !chmod 600 ~/.kaggle/kaggle.json  
  !kaggle datasets download -d moltean/fruits
  '''
  !ls
  with ZipFile(DATASET_ZIP_FILE, 'r') as zipF:
    zipF.extractall()
    print('UnZip Done')

In [0]:
data_dir = 'flower_data/'
PATH = data_dir

train_dir = 'train'
val_dir = 'valid'

In [0]:
batch_size = 8

In [13]:
# again, list total number of classes, and list them all
# os.list dir sorting depends on OS dependent file indexing, so leaving it as it is

classes = os.listdir(f'{data_dir}/{train_dir}')
classes.sort()
ClassesNumer = len(classes)
print("Class Total Count: ", ClassesNumer)
#print(classes)


Class Total Count:  102


In [0]:
# Data augmentation and normalization for training
# Just normalization for validation
data_transforms = {
    train_dir: transforms.Compose([
        transforms.RandomResizedCrop(224),
        transforms.RandomHorizontalFlip(),
        transforms.ToTensor(),
        transforms.Normalize([0.485, 0.456, 0.406], [0.229, 0.224, 0.225])
       # transforms.Normalize((0.5, 0.5, 0.5), (0.5, 0.5, 0.5))
    ]),
    val_dir: transforms.Compose([
        transforms.Resize(224),
        transforms.CenterCrop(224),
        transforms.ToTensor(),
        transforms.Normalize([0.485, 0.456, 0.406], [0.229, 0.224, 0.225])
        #transforms.Normalize((0.5, 0.5, 0.5), (0.5, 0.5, 0.5))
    ]),
}

In [0]:

image_datasets = {x: datasets.ImageFolder(os.path.join(data_dir, x),data_transforms[x]) for x in [train_dir, val_dir]}

dataloaders = {x: torch.utils.data.DataLoader(image_datasets[x], batch_size=16, shuffle=True, num_workers=4) for x in [train_dir, val_dir]}

dataset_sizes = {x: len(image_datasets[x]) for x in [train_dir, val_dir]}

class_names = image_datasets[train_dir].classes

device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")


In [16]:
print(dataloaders)
print(dataset_sizes)
print(device)

{'train': <torch.utils.data.dataloader.DataLoader object at 0x7f5e4e5a9710>, 'valid': <torch.utils.data.dataloader.DataLoader object at 0x7f5e49d3ef98>}
{'train': 6552, 'valid': 818}
cuda:0


In [17]:
print(image_datasets[train_dir])


Dataset ImageFolder
    Number of datapoints: 6552
    Root Location: flower_data/train
    Transforms (if any): Compose(
                             RandomResizedCrop(size=(224, 224), scale=(0.08, 1.0), ratio=(0.75, 1.3333), interpolation=PIL.Image.BILINEAR)
                             RandomHorizontalFlip(p=0.5)
                             ToTensor()
                             Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225])
                         )
    Target Transforms (if any): None


In [18]:
#Let’s visualize a few training images so as to understand the data augmentations.
'''
def imshow(inp, title=None):
    """Imshow for Tensor."""
    inp = inp.numpy().transpose((1, 2, 0))
    mean = np.array([0.485, 0.456, 0.406])
    std = np.array([0.229, 0.224, 0.225])
    inp = std * inp + mean
    inp = np.clip(inp, 0, 1)
    plt.imshow(inp)
    if title is not None:
        plt.title(title)
    plt.pause(0.001)  # pause a bit so that plots are updated


# Get a batch of training data
inputs, classes = next(iter(dataloaders[train_dir]))

# Make a grid from batch
out = torchvision.utils.make_grid(inputs)

imshow(out, title=[class_names[x] for x in classes])
'''

'\ndef imshow(inp, title=None):\n    """Imshow for Tensor."""\n    inp = inp.numpy().transpose((1, 2, 0))\n    mean = np.array([0.485, 0.456, 0.406])\n    std = np.array([0.229, 0.224, 0.225])\n    inp = std * inp + mean\n    inp = np.clip(inp, 0, 1)\n    plt.imshow(inp)\n    if title is not None:\n        plt.title(title)\n    plt.pause(0.001)  # pause a bit so that plots are updated\n\n\n# Get a batch of training data\ninputs, classes = next(iter(dataloaders[train_dir]))\n\n# Make a grid from batch\nout = torchvision.utils.make_grid(inputs)\n\nimshow(out, title=[class_names[x] for x in classes])\n'

###Training the model

Now, let’s write a general function to train a model. Here, we will illustrate:

    Scheduling the learning rate
    Saving the best model

In the following, parameter ''scheduler'' is an LR scheduler object from ''torch.optim.lr_scheduler''.

In [0]:
def train_model(model, criterion, optimizer, scheduler, num_epochs=25):
    since = time.time()

    best_model_wts = copy.deepcopy(model.state_dict())
    best_acc = 0.0

    for epoch in range(num_epochs):
        print('Epoch {}/{}'.format(epoch, num_epochs - 1))
        print('-' * 10)

        # Each epoch has a training and validation phase
        for phase in [train_dir, val_dir]:
            if phase == train_dir:
                scheduler.step()
                model.train()  # Set model to training mode
            else:
                model.eval()   # Set model to evaluate mode

            running_loss = 0.0
            running_corrects = 0

            # Iterate over data.
            for inputs, labels in dataloaders[phase]:
                inputs = inputs.to(device)
                labels = labels.to(device)

                # zero the parameter gradients
                optimizer.zero_grad()

                # forward
                # track history if only in train
                with torch.set_grad_enabled(phase == train_dir):
                    outputs = model(inputs)
                    _, preds = torch.max(outputs, 1)
                    loss = criterion(outputs, labels)

                    # backward + optimize only if in training phase
                    if phase == train_dir:
                        loss.backward()
                        optimizer.step()

                # statistics
                running_loss += loss.item() * inputs.size(0)
                running_corrects += torch.sum(preds == labels.data)

            epoch_loss = running_loss / dataset_sizes[phase]
            epoch_acc = running_corrects.double() / dataset_sizes[phase]

            print('{} Loss: {:.4f} Acc: {:.4f}'.format(
                phase, epoch_loss, epoch_acc))

            # deep copy the model
            if phase == val_dir and epoch_acc > best_acc:
                best_acc = epoch_acc
                best_model_wts = copy.deepcopy(model.state_dict())

        print()

    time_elapsed = time.time() - since
    print('Training complete in {:.0f}m {:.0f}s'.format(
        time_elapsed // 60, time_elapsed % 60))
    print('Best val Acc: {:4f}'.format(best_acc))

    # load best model weights
    model.load_state_dict(best_model_wts)
    return model

In [20]:
# Load a pretrained model and reset final fully connected layer

model_ft = models.densenet201(pretrained='imagenet')
num_ftrs = model_ft.classifier.in_features
model_ft.classifier = nn.Linear(num_ftrs, ClassesNumer)


model_ft = model_ft.to(device)

criterion = nn.CrossEntropyLoss()

# Observe that all parameters are being optimized
optimizer_ft = optim.Adam(model_ft.parameters(),lr=0.001,amsgrad=True)

# Decay LR by a factor of 0.1 every ? epochs
exp_lr_scheduler = lr_scheduler.StepLR(optimizer_ft, step_size=28, gamma=0.1)

/usr/local/lib/python3.6/dist-packages/torchvision/models/densenet.py:212: UserWarning: nn.init.kaiming_normal is now deprecated in favor of nn.init.kaiming_normal_.
  nn.init.kaiming_normal(m.weight.data)


In [21]:
# To view which layers are freeze and which layers are not freezed:
for name, child in model_ft.named_children():
  for name_2, params in child.named_parameters():
    print(name_2, params.requires_grad)

conv0.weight True
norm0.weight True
norm0.bias True
denseblock1.denselayer1.norm1.weight True
denseblock1.denselayer1.norm1.bias True
denseblock1.denselayer1.conv1.weight True
denseblock1.denselayer1.norm2.weight True
denseblock1.denselayer1.norm2.bias True
denseblock1.denselayer1.conv2.weight True
denseblock1.denselayer2.norm1.weight True
denseblock1.denselayer2.norm1.bias True
denseblock1.denselayer2.conv1.weight True
denseblock1.denselayer2.norm2.weight True
denseblock1.denselayer2.norm2.bias True
denseblock1.denselayer2.conv2.weight True
denseblock1.denselayer3.norm1.weight True
denseblock1.denselayer3.norm1.bias True
denseblock1.denselayer3.conv1.weight True
denseblock1.denselayer3.norm2.weight True
denseblock1.denselayer3.norm2.bias True
denseblock1.denselayer3.conv2.weight True
denseblock1.denselayer4.norm1.weight True
denseblock1.denselayer4.norm1.bias True
denseblock1.denselayer4.conv1.weight True
denseblock1.denselayer4.norm2.weight True
denseblock1.denselayer4.norm2.bias Tru

In [22]:
# Train and evaluate
model_ft = train_model(model_ft, criterion, optimizer_ft, exp_lr_scheduler, num_epochs=35)


Epoch 0/34
----------
train Loss: 3.4910 Acc: 0.2021
valid Loss: 2.1118 Acc: 0.4450

Epoch 1/34
----------
train Loss: 1.8460 Acc: 0.4968
valid Loss: 1.1058 Acc: 0.6846

Epoch 2/34
----------
train Loss: 1.2432 Acc: 0.6629
valid Loss: 0.8919 Acc: 0.7628

Epoch 3/34
----------
train Loss: 0.9305 Acc: 0.7445
valid Loss: 0.7791 Acc: 0.7897

Epoch 4/34
----------
train Loss: 0.7506 Acc: 0.7915
valid Loss: 0.4831 Acc: 0.8619

Epoch 5/34
----------
train Loss: 0.6588 Acc: 0.8152
valid Loss: 0.4158 Acc: 0.8985

Epoch 6/34
----------
train Loss: 0.6141 Acc: 0.8298
valid Loss: 0.4327 Acc: 0.8826

Epoch 7/34
----------
train Loss: 0.5265 Acc: 0.8507
valid Loss: 0.4610 Acc: 0.8790

Epoch 8/34
----------
train Loss: 0.4739 Acc: 0.8629
valid Loss: 0.3636 Acc: 0.9059

Epoch 9/34
----------
train Loss: 0.4085 Acc: 0.8826
valid Loss: 0.3535 Acc: 0.9144

Epoch 10/34
----------
train Loss: 0.4100 Acc: 0.8837
valid Loss: 0.2952 Acc: 0.9242

Epoch 11/34
----------
train Loss: 0.3879 Acc: 0.8909
valid Loss

In [23]:
'''
# Generic function to display predictions for a few images
def visualize_model(model, num_images=6):
    was_training = model.training
    model.eval()
    images_so_far = 0
    fig = plt.figure()

    with torch.no_grad():
        for i, (inputs, labels) in enumerate(dataloaders[val_dir]):
            inputs = inputs.to(device)
            labels = labels.to(device)

            outputs = model(inputs)
            _, preds = torch.max(outputs, 1)

            for j in range(inputs.size()[0]):
                images_so_far += 1
                ax = plt.subplot(num_images//2, 2, images_so_far)
                ax.axis('off')
                ax.set_title('predicted: {}'.format(class_names[preds[j]]))
                imshow(inputs.cpu().data[j])

                if images_so_far == num_images:
                    model.train(mode=was_training)
                    return
        model.train(mode=was_training)
'''        

"\n# Generic function to display predictions for a few images\ndef visualize_model(model, num_images=6):\n    was_training = model.training\n    model.eval()\n    images_so_far = 0\n    fig = plt.figure()\n\n    with torch.no_grad():\n        for i, (inputs, labels) in enumerate(dataloaders[val_dir]):\n            inputs = inputs.to(device)\n            labels = labels.to(device)\n\n            outputs = model(inputs)\n            _, preds = torch.max(outputs, 1)\n\n            for j in range(inputs.size()[0]):\n                images_so_far += 1\n                ax = plt.subplot(num_images//2, 2, images_so_far)\n                ax.axis('off')\n                ax.set_title('predicted: {}'.format(class_names[preds[j]]))\n                imshow(inputs.cpu().data[j])\n\n                if images_so_far == num_images:\n                    model.train(mode=was_training)\n                    return\n        model.train(mode=was_training)\n"

In [0]:
#visualize_model(model_ft)

ConvNet as fixed feature extractor: Training only the last fully connected layer
----------------------
Here, we need to freeze all the network except the final layer. We need
to set ``requires_grad == False`` to freeze the parameters so that the
gradients are not computed in ``backward()``.

You can read more about this in the documentation
`here <http://pytorch.org/docs/notes/autograd.html#excluding-subgraphs-from-backward>`__.




In [25]:
# Print the model we just instantiated
print(model_ft)

DenseNet(
  (features): Sequential(
    (conv0): Conv2d(3, 64, kernel_size=(7, 7), stride=(2, 2), padding=(3, 3), bias=False)
    (norm0): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    (relu0): ReLU(inplace)
    (pool0): MaxPool2d(kernel_size=3, stride=2, padding=1, dilation=1, ceil_mode=False)
    (denseblock1): _DenseBlock(
      (denselayer1): _DenseLayer(
        (norm1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
        (relu1): ReLU(inplace)
        (conv1): Conv2d(64, 128, kernel_size=(1, 1), stride=(1, 1), bias=False)
        (norm2): BatchNorm2d(128, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
        (relu2): ReLU(inplace)
        (conv2): Conv2d(128, 32, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
      )
      (denselayer2): _DenseLayer(
        (norm1): BatchNorm2d(96, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
        (relu1): ReLU(inplac

In [26]:
#Print model layers
child_counter = 0
for child in model_ft.children():
   print(" child", child_counter, "is:")
   print(child)
   child_counter += 1

 child 0 is:
Sequential(
  (conv0): Conv2d(3, 64, kernel_size=(7, 7), stride=(2, 2), padding=(3, 3), bias=False)
  (norm0): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
  (relu0): ReLU(inplace)
  (pool0): MaxPool2d(kernel_size=3, stride=2, padding=1, dilation=1, ceil_mode=False)
  (denseblock1): _DenseBlock(
    (denselayer1): _DenseLayer(
      (norm1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (relu1): ReLU(inplace)
      (conv1): Conv2d(64, 128, kernel_size=(1, 1), stride=(1, 1), bias=False)
      (norm2): BatchNorm2d(128, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (relu2): ReLU(inplace)
      (conv2): Conv2d(128, 32, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
    )
    (denselayer2): _DenseLayer(
      (norm1): BatchNorm2d(96, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (relu1): ReLU(inplace)
      (conv1): Conv2d(96, 128, kernel_si

In [27]:
# prints parameters for layers in a pretrained model
for child in model_ft.children():
   for param in child.parameters():
      print(param)
      break
   break

Parameter containing:
tensor([[[[ 0.0259,  0.0389,  0.0367,  ...,  0.0128,  0.0140,  0.0196],
          [-0.0173, -0.0473, -0.0356,  ..., -0.0731, -0.0571, -0.0240],
          [ 0.0270,  0.0337,  0.0444,  ...,  0.1598,  0.1092,  0.0787],
          ...,
          [-0.0220, -0.0984, -0.2217,  ..., -0.4303, -0.3056, -0.1297],
          [-0.0452, -0.0766, -0.0944,  ..., -0.0464, -0.0282, -0.0578],
          [ 0.0196,  0.0484,  0.1013,  ...,  0.1142,  0.0802,  0.0242]],

         [[-0.0054,  0.0104,  0.0009,  ..., -0.0309, -0.0204, -0.0007],
          [-0.0145, -0.0440, -0.0707,  ..., -0.1239, -0.0893, -0.0421],
          [ 0.0453,  0.0808,  0.0999,  ...,  0.2502,  0.1628,  0.0965],
          ...,
          [-0.0518, -0.1297, -0.3206,  ..., -0.5938, -0.4402, -0.2098],
          [-0.1002, -0.1268, -0.1666,  ..., -0.1190, -0.0951, -0.1094],
          [ 0.0074,  0.0655,  0.1293,  ...,  0.1697,  0.1068,  0.0292]],

         [[-0.0019, -0.0009,  0.0149,  ...,  0.0018, -0.0033, -0.0036],
        

In [28]:
'''
model_ft = models.densenet121(pretrained='imagenet')
num_ftrs = model_ft.classifier.in_features
model_ft.classifier = nn.Linear(num_ftrs, ClassesNumer)

model_ft = model_ft.to(device)

criterion = nn.CrossEntropyLoss()

# Observe that all parameters are being optimized
optimizer_ft = optim.SGD(model_ft.parameters(), lr=0.001, momentum=0.9)

# Decay LR by a factor of 0.1 every 7 epochs
exp_lr_scheduler = lr_scheduler.StepLR(optimizer_ft, step_size=7, gamma=0.1)
'''


#Freeze all layers first
for param in model_ft.parameters():
    param.requires_grad = False
    
# Then unfreeze last classification layer only for feature extract
for param in model_ft.classifier.parameters():
    param.requires_grad = True    

    
# To view which layers are freeze and which layers are not freezed:
for name, child in model_ft.named_children():
  for name_2, params in child.named_parameters():
    print(name_2, params.requires_grad)



conv0.weight False
norm0.weight False
norm0.bias False
denseblock1.denselayer1.norm1.weight False
denseblock1.denselayer1.norm1.bias False
denseblock1.denselayer1.conv1.weight False
denseblock1.denselayer1.norm2.weight False
denseblock1.denselayer1.norm2.bias False
denseblock1.denselayer1.conv2.weight False
denseblock1.denselayer2.norm1.weight False
denseblock1.denselayer2.norm1.bias False
denseblock1.denselayer2.conv1.weight False
denseblock1.denselayer2.norm2.weight False
denseblock1.denselayer2.norm2.bias False
denseblock1.denselayer2.conv2.weight False
denseblock1.denselayer3.norm1.weight False
denseblock1.denselayer3.norm1.bias False
denseblock1.denselayer3.conv1.weight False
denseblock1.denselayer3.norm2.weight False
denseblock1.denselayer3.norm2.bias False
denseblock1.denselayer3.conv2.weight False
denseblock1.denselayer4.norm1.weight False
denseblock1.denselayer4.norm1.bias False
denseblock1.denselayer4.conv1.weight False
denseblock1.denselayer4.norm2.weight False
denseblock1.d

In [29]:
# Train and evaluate
#num_ftrs = model_ft.classifier.in_features
#model_ft.classifier = nn.Linear(num_ftrs, ClassesNumer)    

# Observe that all parameters are being optimized
#optimizer_ft = optim.SGD(model_ft.parameters(), lr=0.0001, momentum=0.9)
optimizer_ft = optim.Adam(model_ft.parameters(),lr=0.0001,amsgrad=True)

# Decay LR by a factor of 0.1 every ? epochs
exp_lr_scheduler = lr_scheduler.StepLR(optimizer_ft, step_size=5, gamma=0.1)

model_ft = train_model(model_ft, criterion, optimizer_ft, exp_lr_scheduler, num_epochs=15)

Epoch 0/14
----------
train Loss: 0.1038 Acc: 0.9701
valid Loss: 0.1587 Acc: 0.9731

Epoch 1/14
----------
train Loss: 0.1048 Acc: 0.9701
valid Loss: 0.1594 Acc: 0.9670

Epoch 2/14
----------
train Loss: 0.0978 Acc: 0.9745
valid Loss: 0.1505 Acc: 0.9731

Epoch 3/14
----------
train Loss: 0.1085 Acc: 0.9701
valid Loss: 0.1565 Acc: 0.9645

Epoch 4/14
----------
train Loss: 0.1003 Acc: 0.9725
valid Loss: 0.1586 Acc: 0.9609

Epoch 5/14
----------
train Loss: 0.0965 Acc: 0.9750
valid Loss: 0.1536 Acc: 0.9682

Epoch 6/14
----------
train Loss: 0.0976 Acc: 0.9733
valid Loss: 0.1503 Acc: 0.9682

Epoch 7/14
----------
train Loss: 0.0975 Acc: 0.9728
valid Loss: 0.1595 Acc: 0.9670

Epoch 8/14
----------
train Loss: 0.0960 Acc: 0.9728
valid Loss: 0.1522 Acc: 0.9682

Epoch 9/14
----------
train Loss: 0.0978 Acc: 0.9731
valid Loss: 0.1678 Acc: 0.9645

Epoch 10/14
----------
train Loss: 0.1106 Acc: 0.9718
valid Loss: 0.1581 Acc: 0.9658

Epoch 11/14
----------
train Loss: 0.0971 Acc: 0.9733
valid Loss

In [30]:
# Observe that all parameters are being optimized
optimizer_ft = optim.Adam(model_ft.parameters(),lr=0.0001,amsgrad=True)
model_ft = train_model(model_ft, criterion, optimizer_ft, exp_lr_scheduler, num_epochs=15)

Epoch 0/14
----------
train Loss: 0.1063 Acc: 0.9689
valid Loss: 0.1610 Acc: 0.9670

Epoch 1/14
----------
train Loss: 0.0991 Acc: 0.9713
valid Loss: 0.1546 Acc: 0.9670

Epoch 2/14
----------
train Loss: 0.1114 Acc: 0.9679
valid Loss: 0.1555 Acc: 0.9694

Epoch 3/14
----------
train Loss: 0.0939 Acc: 0.9736
valid Loss: 0.1501 Acc: 0.9694

Epoch 4/14
----------
train Loss: 0.0991 Acc: 0.9725
valid Loss: 0.1529 Acc: 0.9707

Epoch 5/14
----------
train Loss: 0.0924 Acc: 0.9733
valid Loss: 0.1499 Acc: 0.9682

Epoch 6/14
----------
train Loss: 0.0963 Acc: 0.9748
valid Loss: 0.1638 Acc: 0.9645

Epoch 7/14
----------
train Loss: 0.0896 Acc: 0.9731
valid Loss: 0.1597 Acc: 0.9694

Epoch 8/14
----------
train Loss: 0.0925 Acc: 0.9751
valid Loss: 0.1515 Acc: 0.9694

Epoch 9/14
----------
train Loss: 0.0933 Acc: 0.9727
valid Loss: 0.1555 Acc: 0.9694

Epoch 10/14
----------
train Loss: 0.1097 Acc: 0.9689
valid Loss: 0.1543 Acc: 0.9682

Epoch 11/14
----------
train Loss: 0.0991 Acc: 0.9725
valid Loss